In [ ]:
#
#   causticfinder-py calculates caustics based on Hans Witt's Method
#   (PhD thesis, Hamburg, 1991)

#   Copyright 2007, 2008, 2020  Robert W. Schmidt <rschmidt@uni-heidelberg.de>
#
#   This program is free software: you can redistribute it and/or modify
#   it under the terms of the GNU General Public License as published by
#   the Free Software Foundation, either version 3 of the License, or
#   (at your option) any later version.

#   This program is distributed in the hope that it will be useful,
#   but WITHOUT ANY WARRANTY; without even the implied warranty of
#   MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
#   GNU General Public License for more details.
#
#   You should have received a copy of the GNU General Public License
#   along with this program.  If not, see <http://www.gnu.org/licenses/>.


import matplotlib.pyplot as plt
import cmath as cm
import numpy as np

# lens equation
def beta(z):
    return z + z.conjugate()*gamma + (masses/(stars-z).conjugate()).sum()
# poly_fraction1() and poly_fraction2() are
# auxiliary functions for the Newton-Raphson derivative
def poly_fraction1(z,phi):
    sum1=(-2/(stars-z)).sum()
    sum2=(2*masses/(stars-z)**3).sum()
    sum3=(masses/(stars-z)**2).sum()     
    return sum1 + sum2/(gamma-cm.exp(phi*1j)+sum3)
def poly_fraction2(z, phi):
    sum1=(masses/(stars-z)**2).sum()
    sum2=(2*masses/(stars-z)**3).sum()    
    return (sum1 + gamma-cm.exp(phi*1j))/sum2

def causticfinder():

    solns=np.zeros(2*nstars)+np.zeros(2*nstars)*1j
    critical_points=np.zeros(2*nstars*points_per_curve)+np.zeros(2*nstars*points_per_curve)*1j
    caustic_points=critical_points.copy()
 
    zp=0.5+0j
    
    # determine initial roots for phi=0
    for m in range(2*nstars):

        if (m%1000==0):  # print a comment every thousand solutions
            print (m)
            
        zp=zp*cm.exp(0.1j)  # allow Newton-Raphson some distance to converge
        norm=abs(zp)
    
        while (norm>1e-6):
            t=zp

            sum=0
            if (m>0):
                sum=(1/(t-solns[:m])).sum()

            zp = t - 1/(poly_fraction1(t,0)-sum)

            norm=abs(t-zp)
        solns[m]=zp
        #print(m,solns)
        
    # follow curves for phi: 0 -> 2 pi
    for n in range(2*nstars):

        if (n%1000==0): # print a comment every thousand solutions
            print (n)

        zp=solns[n]
        for m in range(points_per_curve):
            t=0
            phi=m*2*cm.pi/points_per_curve
            norm=abs(zp)
    
            while (norm>1e-6):
                t  = zp
                zp = t-poly_fraction2(t,phi)
                norm = abs(t-zp)
            
            z=beta(zp)
            critical_points[n*points_per_curve+m]=zp
            caustic_points[n*points_per_curve+m]=z
    
    return critical_points,caustic_points

In [ ]:
# initialize
nstars = 10
sigma_c = 0.0
gamma = 0.4
gamma = gamma / (1-sigma_c)   # reduced shear
normalize = cm.sqrt(abs(1-sigma_c))
points_per_curve=50           # points per solution branch

star_field='stars.txt'
stars_x, stars_y, masses = np.loadtxt(star_field,unpack=True)

stars=np.zeros(nstars)+np.zeros(nstars)*1j
for i in range(len(stars_x)):
    stars[i]=stars_x[i]*normalize+stars_y[i]*normalize*1j

critical,caustic = causticfinder()

for x in range(2*nstars):
    plt.plot(np.real(caustic[x*points_per_curve:(x+1)*points_per_curve]),
             np.imag(caustic[x*points_per_curve:(x+1)*points_per_curve]),color='blue')
plt.axes().set_aspect('equal')
plt.xlim(-3,3)
plt.ylim(-3,3)